# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/tejo240/.pyenv/versions/3.6.5/bin/python (found version "3.6.5") 
-- Found PythonLibs: /home/tejo240/.pyenv/versions/3.6.5/lib/libpython3.6m.a
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/tejo240/Documents/Facultad/Métodos Numé

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/home/tejo240/Documents/Facultad/Métodos Numéricos/metnum-tp2-20191c/notebooks
Python 3.6.5


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

tar: *.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,Walker Texas Ranger is one of the worst shows ...,neg,938_1.txt
freq,6275,2,6322,2


In [5]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [6]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [8]:
import sentiment

clf = sentiment.KNNClassifier(100)
#esta linea y el siguiente bloque son para correr KNN sin PCA
#esta comentado para poder ejecutar todos los bloques sin ejecutar este
#por equivocacion
#clf.fit(X_train, y_train)


In [9]:
#%%time
#from sklearn.metrics import accuracy_score
#y_pred = clf.predict(X_test)

#acc = accuracy_score(y_test, y_pred)
#print("Accuracy: {}".format(acc))

In [10]:
pca = sentiment.PCA(30)

In [11]:
pca.fit(X_train,10000, 0.000000001)

In [12]:
pca_X_train = pca.transform(X_train, 5)
pca_X_test = pca.transform(X_test,5)

In [13]:
print(pca_X_test.shape[1])
print(pca_X_test)

5
[[ -6.90497268  -2.81010477  -0.93969949  -2.6381538   -2.85025472]
 [ -7.81907144   1.4215357   -1.45152066   2.06105015   0.62981852]
 [ -3.26337631  -1.48763321  -0.44734354  -0.95986524   1.96630703]
 ...
 [-18.35496461  -0.47617457  -2.7777734    4.01547945   3.0803519 ]
 [ -6.3272365   -0.23604173   1.11981217   2.20128377  -0.3660268 ]
 [-18.26258536   0.67115043  -1.17023443  -2.0891063    5.85577905]]


In [14]:
%%time
clf.fit(pca_X_train, y_train)
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

y_pred = clf.predict(pca_X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))



Accuracy: 0.5655776892430279
CPU times: user 2.15 s, sys: 7.71 ms, total: 2.16 s
Wall time: 2.16 s


In [15]:
#All in one. Thuis cell defines the tester functions.
import time
from sklearn.feature_extraction.text import CountVectorizer
def test(pca, max_df_=0.90, min_df_=0.01, max_features_=5000, knn_neighbours_=100, alpha_=30, log=False):

    vectorizer = CountVectorizer(max_df=max_df_, min_df=min_df_, max_features=max_features_)
    vectorizer.fit(text_train)

    X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
    X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

    import sentiment

    clf = sentiment.KNNClassifier(knn_neighbours_)
    #esta linea y el siguiente bloque son para correr KNN sin PCA
    #esta comentado para poder ejecutar todos los bloques sin ejecutar este
    #por equivocacion
    #clf.fit(X_train, y_train)


    pca_X_train = pca.transform(X_train, alpha_)
    pca_X_test = pca.transform(X_test,alpha_)

    #Timing starts, same as previous cell:
    start = time.time()

    clf.fit(pca_X_train, y_train)
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score

    y_pred = clf.predict(pca_X_test)

    acc = accuracy_score(y_test, y_pred)
    pre = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)

    #Timing ends, again, like in previous cell.
    end = time.time()
    delta_time = end - start
    delta_time_str = str(round(delta_time, 2))
    if log:
        comma = ", "
        f = open("../data/test_results/out.csv","a+")
        f.write("\n")
        f.write(str(round(acc,2)) + comma)
        f.write(str(round(pre,2)) + comma)
        f.write(str(round(rec,2)) + comma)
        f.write(delta_time_str + comma)
        f.write(str(round(max_df_,2)) + comma)
        f.write(str(round(min_df_,2)) + comma)
        f.write(str(round(max_features_,2)) + comma)
        f.write(str(round(knn_neighbours_,2)) + comma)
        f.write(str(round(alpha_,2)))
        f.close()

    else:
        print("Time:" + delta_time_str)
        print("Accuracy: {}".format(acc))
        print("Precision: {}".format(pre))
        print("Recall: {}".format(rec))
    return pre


In [16]:
#Actual testing, this cell runs the tester functions and logs or prints results.
#It may take a long time to run.
import numpy as np
#OJO, puse esto en true para testear
test_individually = True
if test_individually:
    max_alpha = 90
    pca = sentiment.PCA(max_alpha)
    pca.fit(X_train,10000, 0.0000000001)
    test(pca, max_df_=0.90, min_df_=0.01, max_features_=5000, knn_neighbours_=100, alpha_=20, log=False)
else:
    max_alpha = 140
    pca = sentiment.PCA(alpha_)
    pca.fit(X_train,10000, 0.0000000001)
    for max_df_ in np.arange(0.90, 0.91, 0.05):
        for min_df_ in np.arange(0.01, 0.02, 0.01):
            for max_features_ in range(5000, 10000, 2000):
                for knn_neighbours_ in range(100, 900, 200):
                    #no se si escribi bien el range, pero la idea es ir del max_alpha hacia abajo asi
                    #te ahorras tener que hacer el fit para cada alpha
                    #(y de hecho saque el fit de la funcion test por este motivo)
                    for alpha_ in range(max_alpha, 30, 30):
                        test(pca, max_df_, min_df_, max_features_, knn_neighbours_, alpha_, log=True)

Time:2.61
Accuracy: 0.6124302788844621
Precision: 0.6155717761557178
Recall: 0.5718437197287698


In [ ]:
def gradient_descent(p0, start_value):
    step0 = 0.01
    step1 = 0.01
    step2 = 500
    step3 = 20
    step4 = 100
    
    old_val = start_value
    max_index = 0
    sign = 1
    new_max=0
    changed = False

    p1 = []        
    
    next_val = test(max(0,p0[0]+step0), p0[1], p0[2], p0[3], p0[4], log=True)
    p1 = [p0[0]+step0, p0[1], p0[2], p0[3], p0[4]]
    if new_max > old_val:
        old_val = new_max
        changed = True
        
    next_val = test(max(0,p0[0]-step0), p0[1], p0[2], p0[3], p0[4], log=True)
    p1 = [p0[0]-step0, p0[1], p0[2], p0[3], p0[4]]
    if new_max > old_val:
        old_val = new_max
        changed = True
    
    next_val = test(p0[0], max(0,p0[1]+step1), p0[2], p0[3], p0[4], log=True)
    p1 = [p0[0], p0[1]+step1, p0[2], p0[3], p0[4]]
    if new_max > old_val:
        old_val = new_max
        changed = True    
    next_val = test(p0[0], max(0,p0[1]-step1), p0[2], p0[3], p0[4], log=True)
    p1 = [p0[0], p0[1]-step1, p0[2], p0[3], p0[4]]
    if new_max > old_val:
        old_val = new_max
        changed = True
        
    next_val = test(p0[0], p0[1], max(0,p0[2]+step2), p0[3], p0[4], log=True)
    p1 = [p0[0], p0[1], p0[2]+step2, p0[3], p0[4]]
    if new_max > old_val:
        old_val = new_max
        changed = True
    next_val = test(p0[0], p0[1], max(0,p0[2]-step2), p0[3], p0[4], log=True)
    p1 = [p0[0], p0[1], p0[2]-step2, p0[3], p0[4]]
    if new_max > old_val:
        old_val = new_max
        changed = True
    
    next_val = test(p0[0], p0[1], p0[2], max(0,p0[3]+step3), p0[4], log=True)
    p1 = [p0[0], p0[1], p0[2], p0[3]+step3, p0[4]]
    if new_max > old_val:
        old_val = new_max
        changed = True
    next_val = test(p0[0], p0[1], p0[2], max(0,p0[3]-step3), p0[4], log=True)
    p1 = [p0[0], p0[1], p0[2], p0[3]-step3, p0[4]]
    if new_max > old_val:
        old_val = new_max
        changed = True
        
    next_val = test(p0[0], p0[1], p0[2], p0[3], max(0,p0[4]+step4), log=True)
    p1 = [p0[0], p0[1], p0[2], p0[3], p0[4]+step4]
    if new_max > old_val:
        old_val = new_max
        changed = True
    
    next_val = test(p0[0], p0[1], p0[2], p0[3], max(0,p0[4]-step4), log=True)
    p1 = [p0[0], p0[1], p0[2], p0[3], p0[4]-step4]
    if new_max > old_val:
        old_val = new_max
        changed = True
        

    
    if not changed:
        f = open("../data/test_results/out.csv","a+")
        f.write("\n")
        f.write("Local maximum found, ending gradient descent")
        f.close()
        exit()
    
    f = open("../data/test_results/out.csv","a+")
    f.write("\n")
    f.write("Exploration finished, starting new one with accuracy:" + str(old_val))
    f.close()
    
    gradient_descent(p1, old_val)
        
p0 = [0.90, 0.01,5000, 100, 30]
gradient_descent(p0,0)
